In [1]:
import IPython

urls = {'collections': 'https://docs.python.org/3/library/collections.html'}

def display_docs(key):
    url = urls[key]
    iframe = '<iframe src=' + url + ' width=1100 height=1000></iframe>'
    return IPython.display.HTML(iframe)

<center>
  <header>
    <h1>Implementing Custom Containers</h1>
    <h3>No, not Docker, Python collections!</h3>
  </header>

<br>
<br>
claus.aichinger@gmail.com
<br>
PyCon UK, 2017-10-26
</center>

### Why are we talking about Containers/Collections?

In general:
- Container/Collections are essential part of every program
- Data structures greatly influence design

In this talk:
- How do I represent my data? (Wrap it in a custom class?)
- How do I implement a custom class mimicking a built-in type? (Implement interface?)

Aim:
- Simple approaches, i.e. the next non-trivial step after using a built-in.
- No meta programming.

### Contents

In short:
- Loads of **double underscore aka dunder methods**

In detail:

- What is a Container/Collection?

- Customization (by examples):
  - Data Representation  
  - Mimicking built-ins

### What is a Container/Collection?

- Container: an object that holds other objects.
- Collection: a sized, iterable container

In [2]:
# more precisely
def is_container(something):
    return hasattr(something, '__contains__')       # allows you to do: item in something

def is_collection(something):
    return (hasattr(something, '__contains__') and 
            hasattr(something, '__iter__') and      # for item in something: ...
            hasattr(something, '__len__'))          #  len(something)

In [3]:
# or rather (this is better):
from collections.abc import Container
from collections.abc import Collection  # <-- Python 3.6

def is_container(something):
    return isinstance(something, Container)

def is_collection(something):
    return isinstance(something, Collection)

In [4]:
# built-in Collections (and therefore Containers)
(is_collection(tuple()), is_collection(list()), 
 is_collection(set()), is_collection(dict()))

(True, True, True, True)

In [5]:
# strings are Collections as well
is_collection('Hi Pythonistas')

True

In [6]:
# things that do not support the Collection protocol
is_container(42), is_container(3.14159), is_container(None)
# neither are module or file object, ...

(False, False, False)

### Take a look at the collections module!

In [7]:
display_docs('collections')  # additional containers

## Customization I (Data Representation)

### Example
Simple data record:

In [8]:
participant = ('Zhang', 'Wei', 58, 70.1)
participant

('Zhang', 'Wei', 58, 70.1)

**Pro**
- `tuple()` is a good choice
  - immutable
  - memory efficient due to **`__slots__`**

**Con**
- What is this data set about?
- Any assertions about the data? If so, who is responsible?

![participant.png](participant.png)

---

**You Ain't Gonna Need It** vs. `tuple()` seems a bit too primitive

#### 1st Iteration: Add Explanation

*a container should be self explaining*

Use:
- **`__repr__`** (how it is displayed in the prompt)
- **`__str__`** (string representation)

---

Let's turn participant into a custimizeable class using **`collections.namedtuple`**

In [9]:
from collections import namedtuple

Participant = namedtuple('Participant', 'given_name, family_name, age, weight')
participant = Participant('Wei', 'Zhang', 58, 70.1)
participant

Participant(given_name='Wei', family_name='Zhang', age=58, weight=70.1)

In [10]:
# help(participant) is still useless

#### 2nd Iteration: Add Documentation

*Docs or it didn't happen - ask Mikey ;)*

Use:
- **`__doc__`**

In [11]:
Participant.__doc__ = "Record class representing participants"
Participant.given_name.__doc__ = "Participant's given name"
Participant.family_name.__doc__ = "Participant's family name"
Participant.age.__doc__ = "Participant's age in years"
Participant.weight.__doc__ = "Participant's age in kg"
# Docs: Changed in version 3.5: Property docstrings became writeable.

#### 3rd Iteration: Support Default Values

Use:
- **`__new__.__defaults__`**  (recall that tuples are immutable, `__init__` won't help, we need `__new__`)

In [12]:
ParticipantWithDefaults = namedtuple('Participant', 
                                     'family_name, given_name, age, weight')
ParticipantWithDefaults.__new__.__defaults__ = (None, None, -1, -1)

In [13]:
participant = ParticipantWithDefaults(given_name='Wei', age=17)
participant

Participant(family_name=None, given_name='Wei', age=17, weight=-1)

#### 4th Iteration: Perform Input Validation

Use:
- **`__new__`**

In [14]:
class ParticipantWithInputValidationAndDefaults(ParticipantWithDefaults):
    
    def __new__(cls, *args, **kwargs):

        args_kwargs = dict(zip(Participant._fields, args))
        args_kwargs.update(kwargs)  # we are lazy here
        
        # validate 'age'
        if args_kwargs['age'] <= 18:
            raise ValueError('age={} but needs to be > 18'.format(args_kwargs['age']))

        self = super(ParticipantWithInputValidationAndDefaults, cls).__new__(cls, **args_kwargs)
        return self

In [15]:
ParticipantWithInputValidationAndDefaults(given_name='Wei', age=17)

ValueError: age=17 but needs to be > 18

In [16]:
ParticipantWithInputValidationAndDefaults(given_name='Wei', age=52)

ParticipantWithInputValidationAndDefaults(family_name=None, given_name='Wei', age=52, weight=-1)

### Summary (Data Representation)

- Default container may bee too primitive and limit flexibility.


- Custom containers open the door to
  - additional functionality when needed
  - default arguments and **input validation**
  - **type hints & static type checking** (via MyPy)


- Provide at least `__repr__`, `__str__` and `__doc__`.


- **`namedtuple`** may be a good starting point for a shallow structure.

---

Take a look at **[attrs: Classes Without Boilerplate](http://www.attrs.org/)**

## Customization II (Mimicking Built-Ins)

Basically

```Python 
from collections.abc import SuitableCollection

class MyCollection(SuitableCollection):
    ...
```

### Example
Data processing task:

```Python

# Could look like so:
def extract_all_answers(df):
    """Cool Stuff ¯\_(ツ)_/¯"""
    return 42 * df

# Iteration/Application:
for path in glob('*.csv'):
    df_in = pd.read_csv(path)
    df_out = function(df_in)
    path_processed = path.replace('.csv', '_processed.csv')
    df_out.to_csv(path_processed)
```

**Pro**
- Obvious what is going on

In [17]:
import pandas as pd
from glob import glob

# Could look like so:
def extract_all_answers(df):
    """Cool Stuff ¯\_(ツ)_/¯"""
    return 42 * df

**Con**
- File handling and iteration should be separated from processing
- Both tasks could be pretty complex, better treat in isolation

---

![data_mapping.png](data_mapping.png)

---

We want:
- read (get) data, write (set) data, iterate over data
- a mapping between input and output (and the related files)

---

Let's implement this behaviour using **`collections.abc.MutableMapping`**.

In [18]:
from collections.abc import MutableMapping

class DataMapping(MutableMapping):
    pass

dm = DataMapping()

TypeError: Can't instantiate abstract class DataMapping with abstract methods __delitem__, __getitem__, __iter__, __len__, __setitem__

In [19]:
from glob import glob
import os
import pandas as pd

def key_from_path(path):
    return os.path.splitext(os.path.basename(path))[0]

class DataMapping(MutableMapping):
    
    def __init__(self, pattern):
        self.pattern = pattern
        # establish in-out-path mappings (to be delegated)
        self.get_paths = {key_from_path(path): path 
                          for path in glob(pattern)}
        self.set_paths = {key: path.replace('data_in', 'data_out') 
                          for key, path in self.get_paths.items()}
    
    def __getitem__(self, key):
        return pd.read_csv(self.get_paths[key])  # to be delegated
    
    def __setitem__(self, key, value):
        value.to_csv(self.set_paths[key])  # to be delegated
        return self

    def __iter__(self):
        return iter(self.get_paths)
                          
    def __len__(self):
        return len(self.get_paths)
                          
    def __delitem__(self, key):
        raise AttributeError('{} does not support item '
                             'deletion'.format(self.__class__.__name__))       

In [20]:
dm = DataMapping('./data/data_in/*.csv')
for key, value in dm.items():
    print(key, ':\n', value, '\n')

a :
    value
0      1
1      2 

b :
    value
0      1
1      2 



In [21]:
for key, df in dm.items():
    dm[key] = extract_all_answers(df)

In [22]:
ls data/data_out

a.csv  b.csv


In [23]:
!cat data/data_out/a.csv

,value
0,42
1,84


### Summary (Mimicking Built-Ins)

- Mimic behavior by inheriting from suitable ABC


- Related methods are derived from base methods


- Direct implementation may be more efficient


- Directly subclassing a built-in can be tricky, you may consider `userlist` or `userdict`


- In general: often sensible to test against interface instead of type

<center>
  <header>
    <h1>Implementing Custom Containers</h1>
    <h3>No, not Docker, Python collections!</h3>
  </header>
  <br/>
  <header>
    <h1>Thank you! Q&A?</h1>
  </header>

<br>
<br>
claus.aichinger@gmail.com
<br>
PyCon UK, 2017-10-26
</center>